In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [3]:
url = 'https://news.daum.net'
# headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0"}
# res = requests.get(url, headers=headers)
# html = res.text
html = urlopen(url)
bs_obj = BeautifulSoup(html, 'html.parser')

### (1) 다음 뉴스 섹션 메뉴와 url 추출

In [4]:
# '홈', '사회', '정치', '경제', '국제', '문화', 'IT', '연재', '포토', '팩트체크'
# 'https://news.daum.net/',
# 'https://news.daum.net/society',
# ....
# 'https://news.daum.nethttps://focus.daum.net/ch/news/factcheck'

- 리스트 생성 : section, link
- section_menu_df 데이터프레임 생성
- daum_news_section.csv 파일로 저장

In [ ]:
daum_news_df

### (2) 섹션별 카드 뉴스에서 제목과 url 추출
- topic, url, section
- 각 섹션의 링크 활용해서 섹션별 카드 뉴스의 제목, 링크, section 저장
- daum_cardNews_link.csv로 저장
- requests로 header를 포함해서 요청하면 한글이 깨짐
    - urlopen()으로 요청/응답

In [67]:
daum_news_df = pd.read_csv("./crawl_data/daum_news_df.csv")
url=daum_news_df["link"][0]
html = urlopen(url)
bs_obj = BeautifulSoup(html, 'html.parser')

In [ ]:
url_sec = bs_obj.select("ul.list_newsheadline2 a")
title_sec = bs_obj.select("ul.list_newsheadline2 strong")
len(url_sec),len(title_sec)
url_sec[1]["href"]
title_sec[1].text

In [ ]:
topic_list = bs_obj.find("ul", {"class":"list_newsheadline2"}).find_all("li")
topic_list[0].find("a")["href"]
topic_list[0].find("a").strong.text

In [ ]:
topic_list = bs_obj.select("ul.list_newsheadline2 li")
topic_list[0].a["href"]
topic_list[0].a.strong.text

In [ ]:
title = []
titleLink = []
for topic in topic_list:
    try :
        title.append(topic.a.strong.text)
        titleLink.append(topic.a["href"])
        section = daum_news_df["menu"][0]
    except:
        print("error 발생")
len(title), len(titleLink), len(section)

In [65]:
daum_section_data = pd.DataFrame({"기사제목":title, "기사내용":titleLink, "분류":section})

In [1]:
# 함수로 작성 : get_topic(url, section)

In [111]:
def get_topic(url, section) :
    html= urlopen(url)
    bs_obj=BeautifulSoup(html, "html.parser")
    if (section=="지식/칼럼") or (section=="연재"):
        topic_list =bs_obj.select("ul.list_newsbasic li")
    else:
        topic_list =bs_obj.select("ul.list_newsheadline2 li")
    
    title=[]
    titleLink=[]
    section
    for topic in topic_list:
        try:
            title.append(topic.a.strong.text)
            titleLink.append(topic.a["href"])
        except:
            pass
    return ({"기사제목":title, "기사내용":titleLink, "분류":section})

In [112]:
# 데이터프레임 생성 : all_topic_df
# topic, url, section

In [ ]:
daum_news_df

In [ ]:
all_topic_df = pd.DataFrame({"기사제목":[], "기사내용":[], "분류":[]})
for i in range(len(daum_news_df)):
    section_topic = pd.DataFrame(get_topic(daum_news_df["link"][i], daum_news_df["menu"][i]))
    all_topic_df = pd.concat([all_topic_df, section_topic], axis = 0, ignore_index=True)
# all_topic_df = pd.DataFrame(get_topic(daum_news_df["link"][0], daum_news_df["menu"][0]))
all_topic_df

In [116]:
# all_topic_df.to_csv("./crawl_data/daum_cardNews_link.csv", index=0, encoding="utf-8-sig")

#### (3) 섹션 헤드라인별 sub 페이지 기사 내용 추출
- 언론사, 제목, 날짜/시간, 기사내용, 작성자

In [6]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [7]:
all_topic_df = pd.read_csv("./crawl_data/daum_cardNews_link.csv")

In [ ]:
# html = urlopen(url)
# bs_obj = BeautifulSoup(html, "html.parser")
# title = bs_obj.select_one("h3.tit_view").text
# bs_obj.select_one("span.txt_info").text
# bs_obj.select_one("span.num_date").text
# bs_obj.select_one("a#kakaoServiceLogo").text
# bs_obj.select_one("div.article_view").text.replace("\n", "")

In [25]:
# 함수로 작성 : get_sub_news_info(url)
def get_sub_news_info(url):
    html = urlopen(url)
    bs_obj = BeautifulSoup(html, "html.parser")
    # headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0"}
    # res = requests.get(url, headers=headers)
    # html = res.text
    title_list, paper_list, datetime_list, writing_list, writer_list = [[] for _ in range(5)]
    try :
        title_list.append(bs_obj.select_one("h3.tit_view").text)
        paper_list.append(bs_obj.select_one("a#kakaoServiceLogo").text)
        datetime_list.append(bs_obj.select_one("span.num_date").text)
        writing_list.append(bs_obj.select_one("div.article_view").text.replace("\n", ""))
        writer_list.append(bs_obj.select_one("span.txt_info").text)
    except Exception as e:
        print("오류 발생 :", e)
    print(len(title_list), len(paper_list), len(datetime_list), len(writing_list), len(writer_list))
    sub_new_dic = {"title":title_list,
                   "paper":paper_list,
                   "datetime":datetime_list,
                   "writing":writing_list,
                   "writer":writer_list}
    return sub_new_dic

In [38]:
# 데이터프레임 생성 : all_sub_news_info_df
# papaer, title, datetime, writing, writer
all_sub_news_info_df = pd.DataFrame({"title":[],
                                     "paper":[],
                                     "datetime":[],
                                     "writing":[],
                                     "writer":[]})
all_sub_news_info_df
for url in all_topic_df["기사내용"]:
    tmp_df = pd.DataFrame(get_sub_news_info(url))
    all_sub_news_info_df=pd.concat([all_sub_news_info_df, tmp_df], axis = 0, ignore_index=True)
all_sub_news_info_df

,title,paper,datetime,writing,writer


1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1


,title,paper,datetime,writing,writer
0,"尹탄핵심판 첫 기일 'D-1'…헌재, 오늘 6인 체제 재판관회의(종합)",연합뉴스,2024. 12. 26. 10:31,출근하는 헌법재판관들 (서울=연합뉴스) 김성민 기자 = 문형배...,황윤기
1,올해는 AI 혁신 확산의 해…新기술로 새해 투자 '청신호',지디넷코리아,2024. 12. 26. 10:16,(지디넷코리아=조이환 기자)한국 경제가 대통령 탄핵정국과 트럼프 2기 정부 출범을 ...,조이환 기자
2,원/달러 환율 장중 1461원 돌파… 2009년 3월 이후 최고,머니S,2024. 12. 26. 10:13,원/달러 환율 강세가 이어진 24일 오전 서울 중구 명동 사설...,이남의 기자
3,"[시선집중] 백령도 주민 ""북풍공작? 주민 목숨을 권력 유지 수단으로 사용하다니.....",MBC,2024. 12. 26. 09:54,■ 방송 : MBC 라디오 표준FM 95.9MHz <김종배의 시선집중>(07:05~...,MBC라디오
4,환율 1460원 수준에 올라…내년 3월까지 안심 못해,아시아경제,2024. 12. 26. 09:52,"원·달러 환율이 최근 4거래일 연속 1450원대로 마감한 가운데, 26일 1460원...",박재현
...,...,...,...,...,...
104,평생 국가·가족 위해 기도하시던… 공명정대 일깨우신 ‘큰 산’[그립습니다],문화일보,2024. 12. 26. 09:03,"아버지(오른쪽 두 번째), 어머니(〃 세 번째)를 모시고 필자...",2024. 12. 26. 09:03
105,임종 때까지 기부 ‘팥죽 할머니’… “나눔은 맛있어요”,문화일보,2024. 12. 24. 11:36,지난 2016년 ‘팥죽 할머니’ 김은숙(오른쪽) 씨와 아들 가...,김지은 기자
106,‘명자 아끼꼬 쏘냐’ ‘만다라’…충무로 시나리오 거장 송길한 84세 별세,중앙일보,2024. 12. 22. 21:08,충무로 시나리오 교과서로 불리는 원로 시나리오 작가 송길한씨가...,나원정
107,“열일곱 평생 뇌전증 투병에도 꽃향기 같았던 삶” [기억저장소],국민일보,2024. 12. 22. 01:02,"2021년 10월 21일 분당차병원에서 심장, 폐, 간, 좌·...",박은주


In [39]:
# 파일로 저장 : daum_sub_news_final.csv
import datetime
today = pd.to_datetime(datetime.datetime.now()).strftime("%Y-%M-%d")
all_sub_news_info_df.to_csv(f"./crawl_data/{today}_daum_sub_news_final.csv", index=0, encoding="utf-8-sig")

In [40]:
all_sub_news_info_df

,title,paper,datetime,writing,writer
0,"尹탄핵심판 첫 기일 'D-1'…헌재, 오늘 6인 체제 재판관회의(종합)",연합뉴스,2024. 12. 26. 10:31,출근하는 헌법재판관들 (서울=연합뉴스) 김성민 기자 = 문형배...,황윤기
1,올해는 AI 혁신 확산의 해…新기술로 새해 투자 '청신호',지디넷코리아,2024. 12. 26. 10:16,(지디넷코리아=조이환 기자)한국 경제가 대통령 탄핵정국과 트럼프 2기 정부 출범을 ...,조이환 기자
2,원/달러 환율 장중 1461원 돌파… 2009년 3월 이후 최고,머니S,2024. 12. 26. 10:13,원/달러 환율 강세가 이어진 24일 오전 서울 중구 명동 사설...,이남의 기자
3,"[시선집중] 백령도 주민 ""북풍공작? 주민 목숨을 권력 유지 수단으로 사용하다니.....",MBC,2024. 12. 26. 09:54,■ 방송 : MBC 라디오 표준FM 95.9MHz <김종배의 시선집중>(07:05~...,MBC라디오
4,환율 1460원 수준에 올라…내년 3월까지 안심 못해,아시아경제,2024. 12. 26. 09:52,"원·달러 환율이 최근 4거래일 연속 1450원대로 마감한 가운데, 26일 1460원...",박재현
...,...,...,...,...,...
104,평생 국가·가족 위해 기도하시던… 공명정대 일깨우신 ‘큰 산’[그립습니다],문화일보,2024. 12. 26. 09:03,"아버지(오른쪽 두 번째), 어머니(〃 세 번째)를 모시고 필자...",2024. 12. 26. 09:03
105,임종 때까지 기부 ‘팥죽 할머니’… “나눔은 맛있어요”,문화일보,2024. 12. 24. 11:36,지난 2016년 ‘팥죽 할머니’ 김은숙(오른쪽) 씨와 아들 가...,김지은 기자
106,‘명자 아끼꼬 쏘냐’ ‘만다라’…충무로 시나리오 거장 송길한 84세 별세,중앙일보,2024. 12. 22. 21:08,충무로 시나리오 교과서로 불리는 원로 시나리오 작가 송길한씨가...,나원정
107,“열일곱 평생 뇌전증 투병에도 꽃향기 같았던 삶” [기억저장소],국민일보,2024. 12. 22. 01:02,"2021년 10월 21일 분당차병원에서 심장, 폐, 간, 좌·...",박은주
